# 라이브러리 불러오기

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
#from scipy.stats import logistic
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import *
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


# 분석에 도움주는 주요 기본 함수 정의
## 메모리 줄여주는 함수 (일반)

In [2]:
import pandas as pd 
import numpy as np

def reduce_mem_usage(df, float16_as32=True):
    # memory_usage()는 데이터프레임의 각 열의 메모리 사용량을 계산하고, sum은 그 값을 합산합니다. B -> KB -> MB로 변환합니다.
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터프레임의 초기 메모리 사용량은 {:.2f} MB입니다.'.format(start_mem))

    for col in df.columns:  # 각 열의 이름을 반복합니다.
        col_type = df[col].dtype  # 열의 데이터 타입을 가져옵니다.
        if col_type != object and str(col_type) != 'category':  # 객체 타입이 아닌 수치형 변수만 처리합니다.
            c_min, c_max = df[col].min(), df[col].max()  # 해당 열의 최소값과 최대값을 계산합니다.
            if str(col_type)[:3] == 'int':  # 정수형 변수일 경우 (int8, int16, int32, int64 모두 포함)
                # 이 열의 값 범위가 int8의 범위에 있을 경우 타입을 int8로 변환합니다. (-128부터 127까지)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 이 열의 값 범위가 int16의 범위에 있을 경우 타입을 int16으로 변환합니다. (-32,768부터 32,767까지)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 이 열의 값 범위가 int32의 범위에 있을 경우 타입을 int32로 변환합니다. (-2,147,483,648부터 2,147,483,647까지)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 이 열의 값 범위가 int64의 범위에 있을 경우 타입을 int64로 변환합니다. (-9,223,372,036,854,775,808부터 9,223,372,036,854,775,807까지)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # 실수형일 경우
                # 값이 float16의 범위에 있을 경우 타입을 변환합니다. 더 높은 정밀도가 필요하면 float32를 고려합니다.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # 더 높은 정밀도가 필요하면 float32를 선택할 수 있습니다.
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                # 값이 float32의 범위에 있을 경우 타입을 float32로 변환합니다.
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # 값이 float64의 범위에 있을 경우 타입을 float64로 변환합니다.
                else:
                    df[col] = df[col].astype(np.float64)
    # 최종 메모리 사용량을 계산합니다.
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 데이터프레임의 메모리 사용량은: {:.2f} MB입니다.'.format(end_mem))
    # 초기 메모리 사용량과 비교하여 몇 퍼센트 감소했는지 계산합니다.
    print('메모리 사용량이 {:.1f}% 감소했습니다.'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## 시간체크함수

In [3]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper

## 데이터 타입 문자열로 바꿔주는 함수

In [4]:
@execution_time
def clean_df(data):
    data['고객ID'] = data['고객ID'].astype(str)
    data = reduce_mem_usage(data)
    return data

# 데이터 불러오기

In [5]:
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
#card_21 = pd.read_csv(DATA_PATH + 'card_21.csv', encoding="euc-kr")
card_22_usage = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")
#card_23 = pd.read_csv(DATA_PATH + 'card_23.csv', encoding="euc-kr")
#customer_21 = pd.read_csv(DATA_PATH_2 + 'customer_21.csv', encoding="euc-kr")
customer_22 = pd.read_csv(DATA_PATH_2 + 'customer_22.csv', encoding="euc-kr")
#customer_23 = pd.read_csv(DATA_PATH_2 + 'customer_23.csv', encoding="euc-kr")

# 데이터 가공

# 22년도 카드 데이터 고객 id 데이터타입 변경

In [6]:
card_22 = clean_df(card_22)
card_22.info()

데이터프레임의 초기 메모리 사용량은 3361.67 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 2881.43 MB입니다.
메모리 사용량이 14.3% 감소했습니다.
Execution time for clean_df: 0h 0m 14.84s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62945850 entries, 0 to 62945849
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   거래년월      int32 
 1   고객ID      object
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 2.8+ GB


In [7]:
customer_22 = clean_df(customer_22)
customer_22.info()

데이터프레임의 초기 메모리 사용량은 2140.22 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 1564.01 MB입니다.
메모리 사용량이 26.9% 감소했습니다.
Execution time for clean_df: 0h 0m 5.53s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21578686 entries, 0 to 21578685
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기준년월      int32 
 1   고객ID      object
 2   연령대       object
 3   성별        object
 4   고객등급      object
 5   자택_시도     object
 6   자택_시군구    object
 7   수신_요구불예금  int32 
 8   수신_거치식예금  int32 
 9   수신_적립식예금  int32 
 10  수신_펀드     int32 
 11  수신_외화예금   int32 
 12  대출금액      int32 
dtypes: int32(7), object(6)
memory usage: 1.5+ GB


## 지역 재분류

In [8]:
# 지역별 그룹을 지정
region_groups = {
    '서울특별시': '수도권', '경기도': '수도권', '인천광역시': '수도권',
    '부산광역시': '영남권', '울산광역시': '영남권', '경상남도': '영남권',
    '충청북도': '충청권', '충청남도': '충청권', '대전광역시': '충청권', '세종특별자치시': '충청권',
    '전북특별자치도': '호남권', '전라남도': '호남권', '광주광역시': '호남권',
    '강원특별자치도': '강원·제주권', '제주특별자치도': '강원·제주권'
}
# '가맹점_광역시도' 열 값을 위의 그룹 기준으로 변환
card_22['가맹점_광역시도'] = card_22['가맹점_광역시도'].replace(region_groups)
# 원본 데이터 변경할 거면 card_2021['가맹점_광역시도'].replace(region_groups, inplace=True)
card_22['가맹점_광역시도'].value_counts()

가맹점_광역시도
대구광역시     35353358
수도권       13083923
경상북도      11029160
영남권        1754131
충청권         845250
강원·제주권      602292
호남권         277736
Name: count, dtype: int64

In [9]:
card_22.head(3)

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202212,3083154411,일반한식,경상북도,경주시,5번이하,20000
1,202212,3557972915,일반한식,경상북도,경주시,5번이하,20000
2,202212,3605837880,일반한식,경상북도,경주시,5번이하,80000


## 카드 데이터 가맹업종명 재분류

In [13]:
# 각 분류명에 해당하는 데이터값을 리스트로 정의
mapping = {
    '교육/학원': ['보습학원', '예·체능계학원', '외국어학원', '기타 교육기관', '기능학원', '학원', '학습지교육', '초중고교육기관'],
    '미용': ['미용원', '피부미용실'],
    '스포츠/레저': ['기타레져업소', '스포츠·레져용품', '스포츠의류', '스크린골프', '헬스클럽', '골프경기장', '종합레져타운', '골프용품 전문점', '골프연습장', '안마/스포츠마사지', '레져업소', '수 영 장'],
    '여행/교통': ['기타숙박업', '철도', '특급호텔', '1급 호텔', '렌터카', '항공사', '콘도', '2급 호텔', '관광여행', '여객선', '고속버스', '택시'],
    '외식': ['일반한식', '주점', '서양음식', '중국식', '일식·회집', '한정식', '칵테일바', '갈비전문점'],
    '대형마트': ['슈퍼마켓', '농협하나로클럽', '농,축협직영매장', '대형할인점'],
    '백화점': ['면 세 점', '자사카드발행백화점'],
    '편의점': ['편 의 점'],
    '음식료품': ['정육점', '스넥', '농·축·수산품', '농축수산가공품', '주류판매점', '홍삼제품', '인삼제품', '기타건강식품'],
    '카페/베이커리': ['기타음료식품', '제과점'],
    '의료/보험': ['약 국', '의 원', '한 의 원', '치과의원', '기타의료기관 및 기타의료기기', '한약방', '한방병원', '생명보험', '건강진단', '기타보험', '병 원(응급실운영)', '치과병원', '동물병원', '종합병원', '손해보험'],
    '주유': ['SK주유소', '쌍용S-OIL', 'GS주유소', '주유소', 'SK가스충전소', '현대정유(오일뱅크)', 'GS가스충전소', '쌍용S-OIL가스충전소', '현대정유가스충전소', 'E1가스충전소', '전기차충전소'],
    '문화생활': ['문화취미기타', '영화관', '티켓'],
    '이동통신요금': ['이동통신요금'],
    '반려동물': ['애완동물'],
    '간편결제': ['인터넷P/G'],
    '기타': ['전자상거래상품권전문판매', '캐주얼의류', 'CATV', '택시회사', '위성방송', '음반영상물', '제 화','주방용구', '연 쇄 점', '액세서리', '전문서적', '주차장', '민예·공예품', '건축용 요업제품', '단란주점', '옷감·직물', '자동차시트·타이어', '독서실', '농기계', '조명기구', '세차장', '사무서비스', '컴퓨터', '가 방', '보일러·펌프·샷시', '카페트,커튼,천막,지물', '자동차부품', '자동차정비', '기타유통업', '이륜차판매', '카인테리어', '화물운송', '내의판매', '목재·석재·철물', '기타 사무용품', '기타가구', '신 발', '비영리/비대상', '기타대인서비스', '기타전기제품', 'DP&E', '주방용식기', '가타자동차서비스', '기타농업관련', '구내매점(국가기관등)', '아동의류', '이용원', '조세서비스', '기념품점', '유흥주점', '통신기기', '화원', '화장품', '기타잡화', '유아원', 'LPG취급점', '페인트', '출판 및 인쇄물', '귀금속', '문구용품', '위탁급식업', '비료,사료,종자', '침구·수예점', '당구장', '가전제품', '인터넷Mall', '가정용품수리', '사무·통신기기수리', '안경', '세탁소', '사우나', '일반서적', '노 래 방', '기계공구',  '완구점', '기타의류', '미곡상', '기타용역서비스', '기타수리서비스', '기타건축자재', '일반가구', '사무용 OA기기', '인테리어전문', '가례서비스업', '시 계', '비영리/대상', '윤활유전문판매', '정수기', '공공요금대행서비스/소득공제비대상', '화방·표구점', '볼 링 장', '인터넷종합Mall', '기타 전문점', '통신판매업1', '종합용역', '전자상거래상품권', '보관및 창고업', '유류판매', '업종미등록', '통신서비스/소득공제비대상', '기타서적문구', '수족관', '공공요금대행서비스/소득공제대상', '상품권전문판매', '혼례서비스업', '냉열기기', '비씨카드 정산용(할인)', '일반(통신판매)', '기타4', '기타1', '부동산중개·임대', 'CATV홈쇼핑', '소프트웨어', '의료용품', '정장', '맞춤복점', '단체복', '상 품 권', 'PG상품권', '비씨카드 정산용가맹점',  '미용재료']
}
# 각 분류에 대해 매핑 (inplace 없이 할당 방식으로 변경)
for category, values in mapping.items():
    card_22['가맹점업종명'] = card_22['가맹점업종명'].replace(values, category)
# 결과 확인
print(card_22['가맹점업종명'].unique())

['외식' '기타' '미용' '스포츠/레저' '의료/보험' '여행/교통' '카페/베이커리' '주유' '대형마트' '음식료품'
 '편의점' '교육/학원' '문화생활' '간편결제' '반려동물' '백화점' '이동통신요금']


In [14]:
card_22.head(10)

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202212,3083154411,외식,경상북도,경주시,5번이하,20000
1,202212,3557972915,외식,경상북도,경주시,5번이하,20000
2,202212,3605837880,외식,경상북도,경주시,5번이하,80000
3,202212,3673581980,외식,경상북도,경주시,5번이하,20000
4,202212,3913986482,외식,경상북도,경주시,5번이하,40000
5,202212,9476232849,기타,경상북도,영천시,5번이하,40000
6,202212,9468456810,외식,경상북도,영천시,5번이하,10000
7,202212,9731736685,외식,경상북도,영천시,5번이하,40000
8,202212,8360876341,외식,경상북도,영천시,5번이하,50000
9,202212,5128581778,외식,경상북도,영천시,5번이하,90000


## 카드 내보내기

In [15]:
card_22.to_csv('card_22_usage.csv', index=False, encoding='utf-8-sig')

## 잔액 0인거 자르기

In [ ]:
customer_21.head(3)

## 연령대 자르기

In [8]:
# 고객 데이터 - 60대 자르기
customer_22_id = customer_22.loc[:, ['고객ID', '연령대']]
customer_22_id.head(3)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상


In [10]:
customer_22_60 = customer_22_id[customer_22_id['연령대'].isin(['60대이상'])]
customer_22_50 = customer_22_id[customer_22_id['연령대'].isin(['50대'])]
customer_22_40 = customer_22_id[customer_22_id['연령대'].isin(['40대'])]
customer_22_60.head()

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상
3,9999989448,60대이상
4,9999988325,60대이상


In [11]:
customer_22_50.head()

,고객ID,연령대
414,9998496340,50대
415,9998491706,50대
416,9998488435,50대
417,9998479714,50대
418,9998477640,50대


In [12]:
customer_22_40.head()

,고객ID,연령대
712,9997377777,40대
713,9997372016,40대
714,9997367955,40대
715,9997366851,40대
716,9997365476,40대


In [13]:
customer_22_40.to_csv('customer_22_40.csv', index=False, encoding='utf-8-sig')

In [14]:
customer_22_50.to_csv('customer_22_50.csv', index=False, encoding='utf-8-sig')

In [15]:
customer_22_60.to_csv('customer_22_60.csv', index=False, encoding='utf-8-sig')

## card, customer merge (join)

### 내보낸거 불러오기

In [5]:
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
#card_21 = pd.read_csv(DATA_PATH + 'card_21.csv', encoding="euc-kr")
card_22_usage = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")
customer_22_40 = pd.read_csv(DATA_PATH_2 + 'customer_22_40.csv', encoding="utf-8-sig")
customer_22_50 = pd.read_csv(DATA_PATH_2 + 'customer_22_50.csv', encoding="utf-8-sig")
customer_22_60 = pd.read_csv(DATA_PATH_2 + 'customer_22_60.csv', encoding="utf-8-sig")
#customer_22_40 = pd.read_csv(DATA_PATH_2 + 'customer_22_40', encoding="utf-8-sig")
#customer_22_50 = pd.read_csv(DATA_PATH_2 + 'customer_22_50', encoding="utf-8-sig")
#customer_22_60 = pd.read_csv(DATA_PATH_2 + 'customer_22_60', encoding="utf-8-sig")
#card_22_usage = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")

In [7]:
customer_22_40.head(6)

,고객ID,연령대
0,9997377777,40대
1,9997372016,40대
2,9997367955,40대
3,9997366851,40대
4,9997365476,40대
5,9997364303,40대


In [8]:
card_22_usage['가맹점업종명'].unique()

array(['일반한식', '자동차정비', '미용원', '기타레져업소', '기념품점', '약 국', '기타숙박업', '기타음료식품',
       '주점', 'SK주유소', '중국식', '슈퍼마켓', '정육점', '스넥', '농·축·수산품', '서양음식',
       '한 의 원', '스포츠·레져용품', '쌍용S-OIL', '안경', '제과점', '화원', '기타잡화', '유아원',
       '편 의 점', 'LPG취급점', '철도', '피부미용실', '페인트', '기계공구', '세탁소', '정장',
       '의 원', '내의판매', '기능학원', '농협하나로클럽', '농,축협직영매장', '귀금속', '문구용품', '사우나',
       '보습학원', '티켓', '기타건축자재', '일식·회집', 'SK가스충전소', '침구·수예점', '골프경기장',
       '자동차부품', 'GS주유소', '구내매점(국가기관등)', '렌터카', '화장품', '인터넷Mall', '완구점',
       '유흥주점', '치과의원', '스포츠의류', '현대정유(오일뱅크)', '주유소', '노 래 방', '스크린골프',
       '인터넷P/G', '자동차시트·타이어', '대형할인점', '조명기구', '가정용품수리', '기타의류', '미곡상',
       '병 원(응급실운영)', '항공사', '기타용역서비스', '가전제품', '문화취미기타', '당구장', '애완동물',
       '기타수리서비스', '주방용구', '연 쇄 점', '액세서리', '전문서적', '한약방', '기타가구', '특급호텔',
       '1급 호텔', '예·체능계학원', '건축용 요업제품', '단란주점', '옷감·직물', '자사카드발행백화점',
       '아동의류', '인삼제품', '목재·석재·철물', '안마/스포츠마사지', '일반가구', '의료용품', '동물병원',
       '농기계', '주차장', '볼 링 장', '세차장', '사무서비스', '일반서적', '컴퓨터', '가 방',

In [9]:
card_22_usage.head()

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202212,3083154411,일반한식,경상북도,경주시,5번이하,20000
1,202212,3557972915,일반한식,경상북도,경주시,5번이하,20000
2,202212,3605837880,일반한식,경상북도,경주시,5번이하,80000
3,202212,3673581980,일반한식,경상북도,경주시,5번이하,20000
4,202212,3913986482,일반한식,경상북도,경주시,5번이하,40000


In [6]:
card_customer_22_40_id= pd.merge(customer_22_40, card_22_usage, on='고객ID', how='inner')
card_customer_22_40_id.head(3)

,고객ID,연령대,거래년월,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,9997364303,40대,202212,이동통신요금,서울특별시,중구,5번이하,70000
1,9997364303,40대,202212,일식·회집,경상북도,영천시,5번이하,120000
2,9997364303,40대,202212,인터넷종합Mall,경기도,성남시 분당구,5번이하,80000


In [ ]:
card_customer_22_40_id.to_csv('cc_22_40.csv', index=False, encoding='utf-8-sig')

In [9]:
card_customer_22_50_id= pd.merge(customer_22_50, card_22_usage, on='고객ID', how='inner')
card_customer_22_50_id.head(30)

MemoryError: Unable to allocate 2.61 GiB for an array with shape (2, 175358665) and data type int64

## 승인건수 정체 알아보기
- 모르겠음

In [ ]:
temp_card_21_sort = temp_card_21.sort_values(by=['고객ID'], ascending=[True])
temp_card_21_sort.head(10)